In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import torch.optim as optim
from transformers import BertForSequenceClassification
import torch.nn.functional as F
import random
from transformers import RobertaTokenizer, RobertaModel

In [ ]:
import os

data = pd.read_csv('final_data.csv')
data.head()

In [ ]:
data = data.dropna(subset=['emt', 'inf'])
# 실수형을 정수형으로 변환하고 1을 빼서 0부터 시작
data['emt'] = data['emt'].astype(int) - 1
data['inf'] = data['inf'].astype(int) - 1

# 레이블 설정
emotional_scores = data['emt']
informational_scores = data['inf']

# "NO"를 제거하는 함수 정의
def remove_no(text):
    if isinstance(text, str):
        return text.replace("NO", "").strip()
    return ""  # NaN 등 문자열이 아닌 값은 빈 문자열로 처리

# 텍스트에서 "NO" 제거하기
data['Empathy'] = data['Empathy'].apply(remove_no)
data['Encouragement'] = data['Encouragement'].apply(remove_no)
data['Contact_Request'] = data['Contact_Request'].apply(remove_no)
data['Post_Advice Requests'] = data['Post_Advice Requests'].apply(remove_no)
data['Solution'] = data['Solution'].apply(remove_no)
data['Link or Reference'] = data['Link or Reference'].apply(remove_no)
data['Experience'] = data['Experience'].apply(remove_no)
data['false_emt'] = data['false_emt'].apply(remove_no)
data['detached_comment'] = data['detached_comment'].apply(remove_no)
data['deep_empathy'] = data['deep_empathy'].apply(remove_no)
data['active_support'] = data['active_support'].apply(remove_no)
data['exp_only'] = data['exp_only'].apply(remove_no)
data['feel_only'] = data['feel_only'].apply(remove_no)
data['ambig_emt'] = data['ambig_emt'].apply(remove_no)
data['weak_advice'] = data['weak_advice'].apply(remove_no)
data['direct_solution'] = data['direct_solution'].apply(remove_no)
data['general_advice'] = data['general_advice'].apply(remove_no)
data['concrete_reference'] = data['concrete_reference'].apply(remove_no)

 



In [ ]:
# 차원축소 [768]
# informational support model

# 랜덤 시드 고정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cpu")
print(f"Using device: {device}")

# BERT 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 실험할 차원 리스트 (조정 가능)
DIMENSIONS = [768]

# Custom Dataset 클래스 정의
class TextDataset(Dataset):
    def __init__(self, text_features, sentiment_scores, labels):
        self.text_features = text_features
        self.sentiment_scores = sentiment_scores
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (
            tuple({key: val[idx] for key, val in text.items()} for text in self.text_features),
            self.sentiment_scores[idx],
            torch.tensor(self.labels[idx], dtype=torch.long),
        )

# BERT 기반 모델 정의 (Sentiment 포함)
class MultiBERTClassifier(nn.Module):
    def __init__(self, hidden_dim, num_classes=3):
        super(MultiBERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")

        # 차원 축소 (5376 → hidden_dim)
        self.fc_reduce = nn.Linear(self.bert.config.hidden_size * 15, hidden_dim)
        self.dropout = nn.Dropout(0.3)

        # Sentiment(2차원) 추가 후 최종 Fully Connected Layer
        self.fc_final = nn.Linear(hidden_dim + 2, num_classes)

    def forward(self, text_inputs, sentiment_scores):
        embeddings = [self.bert(**text).last_hidden_state[:, 0, :] for text in text_inputs]
        combined_features = torch.cat(embeddings, dim=1)  # 5376차원

        reduced_features = self.fc_reduce(combined_features)
        final_features = torch.cat((reduced_features, sentiment_scores), dim=1)

        x = self.dropout(final_features)
        logits = self.fc_final(x)
        return logits

# 데이터 준비
text_columns = [
    "Empathy", "Encouragement", "Contact_Request", "Experience",
    "Solution", "Link or Reference", "Post_Advice Requests",
    "exp_only","feel_only","false_emt","ambig_emt","weak_advice","general_advice","direct_solution", "concrete_reference",
]
text_data = [data[col].tolist() for col in text_columns]
labels = data["inf"].tolist()

# Sentiment 점수 추가 (Min-Max Scaling 적용)
min_val, max_val = -2, 2
comment_sentiments = torch.tensor(data["comment_sentiment"].tolist(), dtype=torch.float32).unsqueeze(1)
post_sentiments = torch.tensor(data["post_sentiment"].tolist(), dtype=torch.float32).unsqueeze(1)

# Sentiment 정규화 (-1 ~ 1 범위)
comment_sentiments = (comment_sentiments - min_val) / (max_val - min_val) * 2 - 1
post_sentiments = (post_sentiments - min_val) / (max_val - min_val) * 2 - 1

sentiment_scores = torch.cat([comment_sentiments, post_sentiments], dim=1).to(device)

# 학습 및 평가 데이터 분리
train_data, eval_data, train_labels, eval_labels, train_sentiments, eval_sentiments = train_test_split(
    list(zip(*text_data)), labels, sentiment_scores, test_size=0.2, stratify=labels, random_state=SEED
)

# 텍스트 토큰화 함수
def tokenize_batch(text_batch):
    return tokenizer(list(text_batch), padding=True, truncation=True, max_length=128, return_tensors="pt")

# 텍스트 데이터를 리스트 형태로 변환 후 토큰화 적용
train_texts = list(map(tokenize_batch, zip(*train_data)))
eval_texts = list(map(tokenize_batch, zip(*eval_data)))

# Dataset 및 DataLoader 생성
train_dataset = TextDataset(train_texts, train_sentiments, train_labels)
eval_dataset = TextDataset(eval_texts, eval_sentiments, eval_labels)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=16, shuffle=False)

# 차원 축소 실험 진행
for dim in DIMENSIONS:
    print(f"\n Running experiment with hidden dimension: {dim}")
    
    # 모델 학습
    model1 = MultiBERTClassifier(hidden_dim=dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model1.parameters(), lr=2e-5)

    # 학습 루프
    epochs = 5
    for epoch in range(epochs):
        model1.train()
        total_loss = 0
        for batch in train_dataloader:
            text_inputs, sentiment_scores, labels = batch

            text_inputs = [{key: val.to(device) for key, val in text.items()} for text in text_inputs]
            sentiment_scores = sentiment_scores.to(device)
            labels = labels.to(device)

            outputs = model1(text_inputs, sentiment_scores)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_dataloader):.4f}")

    # 모델 평가
    model1.eval()
    all_labels, all_predictions, all_probs = [], [], []

    with torch.no_grad():
        for batch in eval_dataloader:
            text_inputs, sentiment_scores, labels = batch

            text_inputs = [{key: val.to(device) for key, val in text.items()} for text in text_inputs]
            sentiment_scores = sentiment_scores.to(device)
            labels = labels.to(device)

            outputs = model1(text_inputs, sentiment_scores)

            probs = F.softmax(outputs, dim=1)
            predictions = torch.argmax(probs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    # 성능 평가
    report = classification_report(all_labels, all_predictions, output_dict=True)
    auc_scores = {f"class_{i}_auc": roc_auc_score((np.array(all_labels) == i).astype(int), np.array(all_probs)[:, i]) for i in range(3)}

    # 결과 정리 및 출력
    report_df = pd.DataFrame(report).T
    auc_df = pd.DataFrame(auc_scores, index=["AUC"]).T
    final_df = pd.concat([report_df.iloc[:-1], auc_df], axis=1)

    print(f"\n 결과 (차원: {dim})")
    print(final_df)


total_indices = list(range(len(data)))
_, eval_indices = train_test_split(
    total_indices, test_size=0.2, stratify=data["inf"].tolist(), random_state=SEED
)

print(len(eval_indices))      
print(len(all_predictions)) 

# 평가 인덱스에 예측값 저장
for i, idx in enumerate(eval_indices):
    data.loc[idx, "predict_inf"] = int(all_predictions[i])

# 예측된 샘플만 추출
predicted_df = data.loc[data["predict_inf"].notna()].copy()
predicted_df["predict_inf"] = predicted_df["predict_inf"].astype(int)

# CSV로 저장
#predicted_df.to_csv("predicted_inf.csv", index=False)

In [ ]:
# 🔹 평가 결과 수집
model1.eval()
label_pred_pairs = []  # (index, true_label, predicted_label)
all_probs = []

with torch.no_grad():
    for batch in eval_dataloader:
        text_inputs, sentiment_scores, labels_batch, indices = batch

        # 디바이스로 이동
        text_inputs = [{key: val.to(device) for key, val in text.items()} for text in text_inputs]
        sentiment_scores = sentiment_scores.to(device)
        labels_batch = labels_batch.to(device)

        # 예측 수행
        outputs = model1(text_inputs, sentiment_scores)
        probs = F.softmax(outputs, dim=1)
        predictions = torch.argmax(probs, dim=1)

        all_probs.extend(probs.cpu().numpy())

        # 🔹 index → pid로 변환하여 저장
        for idx, true_label, pred_label in zip(indices, labels_batch, predictions):
            index = idx.item() if isinstance(idx, torch.Tensor) else idx
            pid_val = data.loc[index, "pid"]  # <- 반드시 유일한 pid
            true_val = true_label.item()
            pred_val = pred_label.item()
            label_pred_pairs.append((pid_val, true_val, pred_val))

# 평가 결과 DataFrame 생성
eval_df = pd.DataFrame(label_pred_pairs, columns=["pid", "true_label", "predict_inf"])

# 입력값 컬럼
input_columns = [
    "pid", "post", "comment",
    "Empathy", "Encouragement", "Contact_Request", "Experience",
    "Solution", "Link or Reference", "Post_Advice Requests",
    "exp_only","feel_only","false_emt","ambig_emt","weak_advice","general_advice","direct_solution", "concrete_reference",
    "comment_sentiment", "post_sentiment"
]


In [ ]:
# 차원축소 실험 [1600]
# emotional support model

# 랜덤 시드 고정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cpu")
print(f"Using device: {device}")

# BERT 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 실험할 차원 리스트 (조정 가능)
DIMENSIONS = [1600]

# Custom Dataset 클래스 정의
class TextDataset(Dataset):
    def __init__(self, text_features, sentiment_scores, labels):
        self.text_features = text_features
        self.sentiment_scores = sentiment_scores
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (
            tuple({key: val[idx] for key, val in text.items()} for text in self.text_features),
            self.sentiment_scores[idx],
            torch.tensor(self.labels[idx], dtype=torch.long),
        )

# BERT 기반 모델 정의 (Sentiment 포함)
class MultiBERTClassifier(nn.Module):
    def __init__(self, hidden_dim, num_classes=3):
        super(MultiBERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")

        # 차원 축소 (5376 → hidden_dim)
        self.fc_reduce = nn.Linear(self.bert.config.hidden_size * 12, hidden_dim)
        self.dropout = nn.Dropout(0.3)

        # Sentiment(2차원) 추가 후 최종 Fully Connected Layer
        self.fc_final = nn.Linear(hidden_dim + 2, num_classes)

    def forward(self, text_inputs, sentiment_scores):
        embeddings = [self.bert(**text).last_hidden_state[:, 0, :] for text in text_inputs]
        combined_features = torch.cat(embeddings, dim=1)  # 5376차원

        reduced_features = self.fc_reduce(combined_features)
        final_features = torch.cat((reduced_features, sentiment_scores), dim=1)

        x = self.dropout(final_features)
        logits = self.fc_final(x)
        return logits

# 데이터 준비
text_columns = [
    "Empathy", "Encouragement", "Contact_Request", "Experience",
    "Solution", "Link or Reference", "Post_Advice Requests",
    "false_emt", "detached_comment", "ambig_emt", "deep_empathy", "active_support"
]
text_data = [data[col].tolist() for col in text_columns]
labels = data["emt"].tolist()

# Sentiment 점수 추가 (Min-Max Scaling 적용)
min_val, max_val = -2, 2
comment_sentiments = torch.tensor(data["comment_sentiment"].tolist(), dtype=torch.float32).unsqueeze(1)
post_sentiments = torch.tensor(data["post_sentiment"].tolist(), dtype=torch.float32).unsqueeze(1)

# Sentiment 정규화 (-1 ~ 1 범위)
comment_sentiments = (comment_sentiments - min_val) / (max_val - min_val) * 2 - 1
post_sentiments = (post_sentiments - min_val) / (max_val - min_val) * 2 - 1

sentiment_scores = torch.cat([comment_sentiments, post_sentiments], dim=1).to(device)

# 학습 및 평가 데이터 분리
train_data, eval_data, train_labels, eval_labels, train_sentiments, eval_sentiments = train_test_split(
    list(zip(*text_data)), labels, sentiment_scores, test_size=0.2, stratify=labels, random_state=SEED
)

# 텍스트 토큰화 함수
def tokenize_batch(text_batch):
    return tokenizer(list(text_batch), padding=True, truncation=True, max_length=128, return_tensors="pt")

# 텍스트 데이터를 리스트 형태로 변환 후 토큰화 적용
train_texts = list(map(tokenize_batch, zip(*train_data)))
eval_texts = list(map(tokenize_batch, zip(*eval_data)))

# Dataset 및 DataLoader 생성
train_dataset = TextDataset(train_texts, train_sentiments, train_labels)
eval_dataset = TextDataset(eval_texts, eval_sentiments, eval_labels)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=16, shuffle=False)

# 차원 축소 실험 진행
for dim in DIMENSIONS:
    print(f"\n Running experiment with hidden dimension: {dim}")
    
    # 모델 학습
    model1 = MultiBERTClassifier(hidden_dim=dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model1.parameters(), lr=2e-5)

    # 학습 루프
    epochs = 5
    for epoch in range(epochs):
        model1.train()
        total_loss = 0
        for batch in train_dataloader:
            text_inputs, sentiment_scores, labels = batch

            text_inputs = [{key: val.to(device) for key, val in text.items()} for text in text_inputs]
            sentiment_scores = sentiment_scores.to(device)
            labels = labels.to(device)

            outputs = model1(text_inputs, sentiment_scores)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_dataloader):.4f}")

    # 모델 평가
    model1.eval()
    all_labels, all_predictions, all_probs = [], [], []

    with torch.no_grad():
        for batch in eval_dataloader:
            text_inputs, sentiment_scores, labels = batch

            text_inputs = [{key: val.to(device) for key, val in text.items()} for text in text_inputs]
            sentiment_scores = sentiment_scores.to(device)
            labels = labels.to(device)

            outputs = model1(text_inputs, sentiment_scores)

            probs = F.softmax(outputs, dim=1)
            predictions = torch.argmax(probs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    # 성능 평가
    report = classification_report(all_labels, all_predictions, output_dict=True)
    auc_scores = {f"class_{i}_auc": roc_auc_score((np.array(all_labels) == i).astype(int), np.array(all_probs)[:, i]) for i in range(3)}

    # 결과 정리 및 출력
    report_df = pd.DataFrame(report).T
    auc_df = pd.DataFrame(auc_scores, index=["AUC"]).T
    final_df = pd.concat([report_df.iloc[:-1], auc_df], axis=1)

    print(f"\n 결과 (차원: {dim})")
    print(final_df)
